In [1]:
import numpy as np

In [2]:
#!pip install datasets
from datasets import load_dataset
trainingDataset = load_dataset("wmt16","de-en", split="train[:50000]")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [3]:
validationDataset = load_dataset("wmt16","de-en", split="validation")
testingDataset = load_dataset("wmt16","de-en", split="test")

In [4]:
from transformers import T5Tokenizer
tokenizer = T5Tokenizer.from_pretrained("google-t5/t5-small")

source_lang = "en"
target_lang = "de"
prefix = "translate English to German: "


def preprocess_function(examples):
    inputs = [prefix + example[source_lang] for example in examples["translation"]]
    targets = [example[target_lang] for example in examples["translation"]]
    model_inputs = tokenizer(inputs, text_target=targets, max_length=128, truncation=True)
    return model_inputs

tokenizedTrainingData = trainingDataset.map(preprocess_function, batched=True)
tokenizedValidationData = validationDataset.map(preprocess_function, batched=True)
tokenizedTestingData = testingDataset.map(preprocess_function, batched=True)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
from transformers import DataCollatorForSeq2Seq
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model="google-t5/t5-small", return_tensors="tf")

In [11]:
from transformers import AdamWeightDecay
optimizer = AdamWeightDecay(learning_rate=2e-5, weight_decay_rate=0.01)
from transformers import TFAutoModelForSeq2SeqLM
model = TFAutoModelForSeq2SeqLM.from_pretrained("google-t5/t5-small")

All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [12]:
tf_train_set = model.prepare_tf_dataset(
    tokenizedTrainingData,
    shuffle=True,
    batch_size=8,
    collate_fn=data_collator,
)

In [13]:
import tensorflow as tf
model.compile(optimizer=optimizer)

In [ ]:
history = model.fit(x=tf_train_set, epochs=3)

Epoch 1/3
1851/6250 [=======>......................] - ETA: 9:59 - loss: 1.3011

In [ ]:
prefix = "translate English to German:"
inputs = [prefix + example['en'] for example in validationDataset["translation"]]
tokenizedValidationData = tokenizer(inputs, return_tensors="tf", padding = True).input_ids

In [11]:
outputs = model.generate(tokenizedValidationData, do_sample=True)

/usr/local/lib/python3.10/dist-packages/transformers/generation/tf_utils.py:837: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length.  recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [15]:
translatedSentences = []
for output in outputs:
    translatedSentences.append(tokenizer.decode(output, skip_special_tokens=True))

In [16]:
translatedTrue = []
for sent in validationDataset["translation"]:
    translatedTrue.append(sent["de"])

In [22]:
#!pip install evaluate
#!pip install bert_score
from datasets import load_metric
from evaluate import load
metric = load("bleu")
bleu_score = metric.compute(predictions=translatedSentences, references=translatedTrue)
meteor = load('meteor')
meteor_score = meteor.compute(predictions=translatedSentences, references=translatedTrue)
bertscore = load("bertscore")
bertscore_score = bertscore.compute(predictions=translatedSentences, references=translatedTrue, lang = "en")

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


OutOfMemoryError: CUDA out of memory. Tried to allocate 198.00 MiB. GPU 0 has a total capacity of 14.75 GiB of which 7.06 MiB is free. Process 17562 has 14.74 GiB memory in use. Of the allocated memory 1015.24 MiB is allocated by PyTorch, and 4.76 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [21]:
import torch
torch.cuda.empty_cache()

In [26]:
checkpoint_filepath = '/content/drive/My Drive/Assignment 3 NLP/Task2 Part C checkpoint.pth'
model.save_weights(checkpoint_filepath)